In [11]:
import paramiko, re
from getpass import getpass

#create pass prompt
thepass = getpass(prompt="Please enter SSH Password")

#host information
host = "192.168.6.71"
port = 2222
#username = test 
#password = password
username = "david.thomsen"
password = thepass

try:
    
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)
    
except paramiko.AuthenticationException:
    print("auth failed")

Uploading Kraken

In [12]:
sftp = ssh.open_sftp()

ssh.exec_command('cd /home/david.thomsen/')

sftp.put('kraken', "kraken")

<SFTPAttributes: [ size=0 uid=1286604565 gid=1286603265 mode=0o100644 atime=1667248526 mtime=1667256153 ]>

Manually Run Commands

In [14]:
sftp = ssh.open_sftp()



stdin, stdout, stderr = ssh.exec_command('ls -a /home')


#print(stdin.readlines())
print(stdout.readlines())
#print(stderr.readlines())

['.\n', '..\n', '.d.log\n', 'Acese1965\n', 'Andneas\n', 'Andoged\n', 'Asereardscon\n', 'Denestomed\n', 'Droutich1970\n', 'Enten1988\n', 'Estion59\n', 'Gulay1979\n', 'Hanceek\n', 'Haremanse84\n', 'Holow1939\n', 'Houred\n', 'Husires\n', 'Madve1942\n', 'Minat1995\n', 'Nowers\n', 'Obbire\n', 'Peding\n', 'Saidess\n', 'Scouned\n', 'Swout1992\n', 'Theame\n', 'Trae1989\n', 'Unat1986\n', 'Wagood71\n', 'Weirche\n', 'Whishatim\n', 'Whitu1992\n', 'abijah.buttendorf\n', 'alexander.brown\n', 'asani.vessup\n', 'benjamin.gifford\n', 'blaise.notter\n', 'christopher.mchugh\n', 'ciaran.byrne\n', 'david.thomsen\n', 'duane.dunston\n', 'dylan.navarro\n', 'eli.minkoff\n', 'emily.crawford\n', 'ethan.deane\n', 'hasan.hashim\n', 'jack.leonard\n', 'jacob.mayotte\n', 'jahseem.maxwell\n', 'john.tiseo\n', 'max.gallagher\n', 'michael.pinelli\n', 'miles.campbell\n', 'noah.stiles\n', 'p.dalton\n', 'paul.gleason\n', 'reed.simon\n', 'samuel.barrows\n', 'samuel.johnson01\n', 'sarah.fornaldes\n', 'sebastian.meredith\n', '

Run Kraken as a sudo user and save PIDs in krakenout.txt

In [15]:
stdin, stdout, stderr = ssh.exec_command("sudo -S /home/ubuntu/kraken --folder /usr/bin --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin")

stdin.write(thepass + '\n')
stdin.flush()

#print(stdout.readlines())

pidlist = []

for line in stdout.readlines(): 
    if re.findall(r'.*pid.*', line):
        pidlist.append(line.split(" ")[-2].split("=")[-1])
        
        # was going to use a seperate doc for pids, ended up using a list
        #with open('krakenout.txt', 'a') as k:
        #    k.write(line.split(" ")[-2].split("=")[-1] + ' ')

print(pidlist)


['519423', '561559']


Executing lsof using PIDs from above


In [42]:
filelist = []

for i in pidlist:
    stdin, stdout, stderr = ssh.exec_command("sudo -S lsof -p " + str(i) + " | awk '{print $9}'")

    stdin.write(thepass + '\n')
    stdin.flush()

    print("Process ID: " + i + '\n')
    ff = (stdout.readlines())
    print(ff)
    print('\n')
    
    pidout = ff[3:]
    
    for f in pidout:
        filenames = (f.split('/')[-1])
        #print(filenames)
        
        if ".so\n" in filenames:
            filelist.append(filenames)

print(filelist)




Process ID: 519423

['NAME\n', '/\n', '/\n', '/snap/snapd/17336/usr/lib/snapd/snapd\n', '/usr/lib/x86_64-linux-gnu/libresolv-2.31.so\n', '/usr/lib/x86_64-linux-gnu/libnss_dns-2.31.so\n', '/usr/lib/x86_64-linux-gnu/libnss_files-2.31.so\n', '/usr/lib/x86_64-linux-gnu/libc-2.31.so\n', '/usr/lib/x86_64-linux-gnu/libpthread-2.31.so\n', '/usr/lib/x86_64-linux-gnu/ld-2.31.so\n', '/dev/null\n', 'type=STREAM\n', 'type=STREAM\n', 'type=STREAM\n', '/run/snapd.socket\n', '[eventpoll]\n', '/var/lib/snapd/state.lock\n', '/run/snapd-snap.socket\n', '\n', 'pipe\n', 'pipe\n']


Process ID: 561559

['NAME\n', '/home/p.dalton\n', '/\n', '/usr/local/bin/egrep\n', '/usr/lib/x86_64-linux-gnu/libc-2.31.so\n', '/usr/lib/x86_64-linux-gnu/libpthread-2.31.so\n', '/usr/lib/x86_64-linux-gnu/ld-2.31.so\n', '/dev/null\n', '/home/p.dalton/.d.log\n', '/home/p.dalton/.d.log\n', '/dev/null\n', '/home/p.dalton/d.pid\n', '*:http-alt\n', '[eventpoll]\n', 'pipe\n', 'pipe\n']


['libresolv-2.31.so\n', 'libnss_dns-2.31.so\n',

Using the Results above, download the suspicious files. 

In [60]:
sftp = ssh.open_sftp()

for file in filelist:
    sftp.get(file, 'Week7\Classwork\SusFiles')


FileNotFoundError: [Errno 2] No such file or directory: 'Week7\\Classwork\\SusFiles'

5


In [71]:
for i in pidlist:
    stdin, stdout, stderr = ssh.exec_command("sudo -S lsof -p " + str(i))

    stdin.write(thepass + '\n')
    stdin.flush()

    print("Process ID: " + i + '\n')
    ez = (stdout.readlines())
    print(ez)
    print('\n')
    
stdin, stdout, stderr = ssh.exec_command("ls -a /home/p.dalton")

for x in stdout.readlines():
    print(x)


SSHException: SSH session not active

What is the process name?

egrep

Which user is running the process?

p.dalton

Has the user logged in before via SSH?

Yes

When did the user login?

Last Login was Oct 25 at 11:32 PM

How long was the login session?

9 min

What is located in the unauthorized user’s home directory?

aqwerpuqwerj, d.pid, test-lk

Download any suspicious files.


Based on what you discovered and the information in the threat report, what is your conclusion about the host you analyzed?

The host we analyzed does seem to have a breach with the user p.dalton running unknonwn commands through SSH as well as having multiple suspicious files within their home dir. 

What did you like the most and least about this assignment?

I liked being able to work with ssh through python although it was very tedious and difficult to manuever. 
I disliked how the lab was worded and wish it was both a bit more clear with both the wording as well as what it would like us to do. 

In [ ]:
ssh.close